# Connect Three 

The primary description of this coursework is available on the CM20252 Moodle page. This is the Jupyter notebook you must complete and submit to receive marks. This notebook adds additional detail to the coursework specification but does not repeat the information that has already been provided there. 

You must follow all instructions given in this notebook precisely.

Restart the kernel and run all cells before submitting the notebook. This will guarantee that we will be able to run your code for testing. Remember to save your work regularly.

__You will develop players for Connect-Three on a grid that is 5 columns wide and 3 rows high. An example is shown below showing a win for Player Red.__

<img src="images/connect3.png" style="width: 200px;"/>

## Preliminaries

For your reference, below is a visual depiction of the agent-environment interface in reinforcment learning. The interaction of the agent with its environments starts at decision stage $t=0$ with the observation of the current state $s_0$. (Notice that there is no reward at this initial stage.) The agent then chooses an action to execute at decision stage $t=1$. The environment responds by changing its state to $s_1$ and returning the numerical reward signal $r_1$. 

<img src="images/agent-environment.png" style="width: 500px;"/>

<br><br><br>

Below, we provide some code that will be useful for implementing parts of this interface. You are not obligated to use this code; please feel free to develop your own code from scratch. 

### Code details

We provide a `Connect` class that you can use to simulate Connect-Three games. The following cells in this section will walk you through the basic usage of this class by playing a couple of games.

We import the `connect` module and create a Connect-Three environmnet called `env`. The constructor method has one argument called `verbose`. If `verbose=True`, the `Connect` object will regularly print the progress of the game. This is useful for getting to know the provided code, debugging your code, or if you just want to play around. You will want to set `verbose=False` when you run hundreds of episodes to complete the marked exercises.

This `Connect` environment uses the strings `'o'` and `'x'` instead of different disk colors in order to distuingish between the two players.

Before we start a game, we should call the `reset()` method. This method cleans the board and resets other state variables. The `first_player` argument can be specified (`'o'` or `'x'`) to deterministically choose the starting player. It defaults to `"random"`, in which case each player starts the game with probability of $\frac{1}{2}$.

In [7]:
import connect
env = connect.Connect(verbose=True)
env.reset(first_player='o')

Game has been reset.
[[' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']]


We can interact with the environment using the `act()` method. This method takes an `action` as input and computes the response of the environment. An action is defined as the column index that a disk is dropped into.

In [8]:
env.act(action=2)

[[' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'o' ' ' ' ']]


The `act()` method inserts a disk into the specified column. 

If we want to change the player on move, we can do so by using the `change_turn()` method. We can check whose turn it is by accessing the `.player_at_turn` attribute.

In [9]:
print("Current player at turn:", env.player_at_turn)
env.change_turn()
print("Current player at turn:", env.player_at_turn)

# Drop another disk into the same centre column.
env.act(action=2)

Current player at turn: o
Current player at turn: x
[[' ' ' ' ' ' ' ' ' ']
 [' ' ' ' 'x' ' ' ' ']
 [' ' ' ' 'o' ' ' ' ']]


Because we set `verbose=True`, the grid is printed each time we call the `act()` method. This grid is stored as a two-dimensional numpy array in the connect class and you can easily access by calling...

In [10]:
current_grid = env.grid
print(current_grid)
# Notice that the grid now appears to be "upside down" because numpy arrays are printed from "top to bottom".

[[' ' ' ' 'o' ' ' ' ']
 [' ' ' ' 'x' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ']]


In [11]:
# Let's make another move.
env.change_turn()
env.act(action=2)

[[' ' ' ' 'o' ' ' ' ']
 [' ' ' ' 'x' ' ' ' ']
 [' ' ' ' 'o' ' ' ' ']]


If we make another move with `act(action=2)`, the environment will throw an error because that column is already filled.

In [12]:
# This cell should throw an error!
env.change_turn()
env.act(action=2)

IndexError: index 3 is out of bounds for axis 0 with size 3

The attribute `.available_actions` contains a numpy array of all not yet filled columns.

In [13]:
print(env.available_actions)
# Column index '2' is missing because this column is already filled

[0 1 3 4]


The `Connect` class contains a method called `was_winning_move()` that checks whether the last move won the game (returns `True`) or not (returns `False`). 

In [14]:
# Obviously the game has not yet been won by any player.
print("Winning move?", env.was_winning_move()) 

# Make some moves
env.act(action=3)
env.change_turn()
env.act(action=1)
env.change_turn()
env.act(action=3)
env.change_turn()
env.act(action=0)

# Check again whether a player has won the game.
print("Winning move?", env.was_winning_move()) 

Winning move? False
[[' ' ' ' 'o' ' ' ' ']
 [' ' ' ' 'x' ' ' ' ']
 [' ' ' ' 'o' 'x' ' ']]
[[' ' ' ' 'o' ' ' ' ']
 [' ' ' ' 'x' ' ' ' ']
 [' ' 'o' 'o' 'x' ' ']]
[[' ' ' ' 'o' ' ' ' ']
 [' ' ' ' 'x' 'x' ' ']
 [' ' 'o' 'o' 'x' ' ']]
[[' ' ' ' 'o' ' ' ' ']
 [' ' ' ' 'x' 'x' ' ']
 ['o' 'o' 'o' 'x' ' ']]
Player ' o ' has won the game!
Winning move? True


Finally, the `Connect` class contains a method called `grid_is_full()` that checks whether the grid still contains empty slots. You can use this method to check whether the game is a draw.

Feel free to modify existing or add new methods to the `Connect` class.

## Part 1: Q-learning

Your opponent is always the first player. Your agent is always the second player.

For your reference, the pseudo-code for Q-learning is reproduced below from the textbook (Reinforcement Learning, Sutton & Barto, 1998, Section 6.5).
<img src="images/q_learning.png" style="width: 600px;"/>

Prepare a **learning curve** following the directions below. We refer to this as Plot 1.

After $n$ steps of interaction with the environment, play $m$ games with the current policy of the agent (without modifying the policy). Think of this as interrupting the agent for a period of time to test how well it has learned so far. Your plot should show the total score obtained in these $m$ games as a function of $n, 2n, 3n, … kn$. The choices of $n$ and $k$ are up to you. They should be reasonable values that demonstrate the efficiency of the learning and how well the agent learns to play the game eventually. Use $m=10$. 

This plot should show the mean performance of $a$ agents, not the performance of a single agent. Because of the stochasticity in the environment, you will obtain two different learning curves from two different agents even though they are using exactly the same algorithm. We suggest setting $a$ to 30 or higher.

Present a single mean learning curve with your choice of parameters $\epsilon$ and $\alpha$. The plot should also show (as a baseline) the mean performance of a random agent that does not learn but chooses actions uniformly randomly from among the legal actions. Label this line “Random Agent”. 

Please include this plot as a static figure in the appropriate cell below. You can look at the source code of this markdown cell to find out how to embed figures using html or you can use drag & drop. If you link to locally stored images, make sure to include those in your submission.

In [205]:
import connect
import random
import numpy as np
env = connect.Connect(verbose=False)
q = {}

def string_state(state):
    
    """Turns current game grid into a string"""
    
    stringed = ""
    
    for i in range(0,3):
        for j in range(0,5):
            stringed += state[i,j]
            
    return stringed

def get_max(state_actions):
    s = string_state(env.grid)
    if s not in q:
        l = {}
        for i in range(0, len(state_actions)):
            l[str(state_actions[i])] = 0.0
        q[s] = l
        return 0
    else:
        avs = q[s]
        
        highest = -10000
        
        for i in avs:
            if avs[i] > highest:
                highest = avs[i]
                
        return highest
    
def get_greedy():
    actions = env.available_actions
    s = string_state(env.grid)
    if s not in q:
        l = {}
        for i in range(0, len(state_actions)):
            l[str(state_actions[i])] = 0.0
        q[s] = l
        
        # returns position in array of available actions
        return random.randint(0, len(av)-1)
    else:
        avs = q[s]
        #print("avs", avs)
        #print("available", actions)
        
        highest = -10000
        choice = 0
        
        iteration = 0
        for i in avs:
            if avs[i] > highest:
                highest = avs[i]
                choice = iteration               
            iteration += 1
                            
        #print("choice:",choice)
        return choice

def update_q(state,last_action,reward,state_actions):
    
    alpha = 0.7
    gamma = 0.05
    
    s = string_state(state)
    
    if s not in q:
        l = {}
        for i in range(0, len(state_actions)):
            j = str(state_actions[i])
            l[j] = 0.0
        q[s] = l
    else:
        avs = q[s]
        max_a = get_max(state_actions)
        #print(max_a)
        
        a = str(last_action)
        if a == '0':
            avs['0'] += float(alpha * (reward + gamma * max_a - avs['0']))
        elif a == '1':
            avs['1'] += float(alpha * (reward + gamma * max_a - avs['1']))
        elif a == '1':
            avs['2'] += float(alpha * (reward + gamma * max_a - avs['2']))
        elif a == '3':
            avs['3'] += float(alpha * (reward + gamma * max_a - avs['3']))
        elif a == '4':
            avs['4'] += float(alpha * (reward + gamma * max_a - avs['4']))

def perform_random_move():
    """Performs a random move for whatever player called it"""
    av = env.available_actions
    action = random.randint(0, len(av) - 1)
    env.act(av[action])
    
def player_turn(testing):
    env.change_turn()
    av = env.available_actions
        
    if not testing:
        action = random.randint(0, len(av) - 1)
        env.act(av[action])
    else:
        #action = random.randint(0, len(av) - 1)
        action = int(get_greedy())
        #print("action:",action)
        
        #print(env.available_actions)
        if not action in env.available_actions:
            #print("not")
            action = random.randint(0, len(env.available_actions) - 1)
            env.act(av[action])
        else:
            env.act(action)
            return action
    
    #perform_random_move()
        
    return av[action]
    
def opponent_turn():
    env.change_turn()
    perform_random_move()

def play_turn_and_done(testing):
    # check if grid is full before performing player term
    
    reward = 0
    
    if env.grid_is_full(): return 0
    
    state = np.copy(env.grid)
    state_actions = np.copy(env.available_actions)
    last_action = player_turn(testing)
    
    # if player did winning turn then return 1 (for win)
    if(env.was_winning_move()):
        reward = 1
    else:
        opponent_turn()
    
        # if opponent bot won then return -1 (for loss)
        if(env.was_winning_move()):
            reward = -1
            
    update_q(state,last_action,reward,state_actions)
    
    if reward == 1 or reward == -1:
        return reward
    
    return -2
    

def play_game(testing):
    env.reset(first_player='o')
    done = False
    perform_random_move()
    while(not done):
        ret = play_turn_and_done(testing)
        if ret == 1 or ret == 0 or ret == -1:
            return ret

def game():
    print("Playing game...")
    
    #unique_states = 0
    
    games = 25000
    games_test = 100
    games_won = 0
    games_lost = 0
    
    fifth = games/5
    two_fifths = 2 * fifth
    three_fifths = 3 * fifth
    four_fifths = 4 * fifth
    
    for i in range(0,games): 
        
        if i == fifth : print("20% done...")
        if i == two_fifths: print("40% done...")
        if i == three_fifths: print("60% done...")
        if i == four_fifths: print("80% done...")
            
            
        result = play_game(testing=False)

            
    for j in range(0,100):
        result = play_game(testing=True)
        if result == 1:
            games_won += 1
        elif result == -1:
            games_lost += 1
            
        #print("Wins:", games_won)
        #print("Losses:", games_lost)
        #print("Draws:", (j - games_won - games_lost + 1))
            
    
    
    print(q)
    print("Done!")
    print("Average Win Percentage for random: ~36-37%")
    print("Average win % if just increment q(s,a) by 1 everytime it's mentioned: ~56-58%")
    print("Win Percentage:", (100 * games_won/games_test),"%")
    print("Loss Percentage:", (100 * games_lost/games_test),"%")
    print("Draw Percentage:", (100 * (games_test - games_won - games_lost)/games_test), "%")
    
    
def list_test():
    l = {}

In [206]:
game()

Playing game...
20% done...
40% done...
60% done...
80% done...
{'   o           ': {'0': 4.7080664828417514e-05, '1': 5.070511387178888e-05, '2': 0.0, '3': 2.7975389683734405e-06, '4': 0.0021521936441208887}, 'o xo           ': {'0': 5.778562798464827e-05, '1': 0.0005084321854511354, '2': 0.0, '3': 0.00038020027991495306, '4': 0.00104945645825693}, 'oxxoo          ': {'0': 0.00124744701025, '1': 0.04888407936320216, '2': 0.0, '3': 0.004389583030229583, '4': 0.0012150775}, 'oxxoo x o      ': {'0': -0.27233850000000004, '1': 0.973, '2': 0.0, '3': 0.031849999999999996, '4': -0.7}, ' o             ': {'0': 0.0003505212101884429, '1': 8.686014561970515e-07, '2': 0.0, '3': 0.00034058148779106596, '4': 0.00021568002535480175}, ' oxo           ': {'0': 0.0014004780652291278, '1': 8.891268814625571e-05, '2': 0.0, '3': 0.0017199347958089003, '4': 0.0001885050194256988}, 'xoxo o         ': {'0': 0.036981034999999995, '1': 0.0022050000000000004, '2': 0.0, '3': 6.924159822125001e-05, '4': 9.217010

(A) [continued} Insert your static learning curve here (Plot 1).

YOUR ANSWER HERE 

(B) In 3 sentences or less, explain your conclusions from the plot above. How close does your (average) agent get to the best possible level of performance? How efficiently does your (average) agent learn? 

YOUR ANSWER HERE 

(C) In five sentences or less, explain the key aspects of your implementation. How many state-action pairs do you represent in your Q-table? Describe and justify your settings of $\alpha$ and $\epsilon$. Are there any things you tried out that are not in your final implementation?

YOUR ANSWER HERE

(D) In the cell below, make it possible for us to produce from scratch a learning curve similar to Plot 1 but for a single agent, for a $k$ value of your own choosing. You do not need to include the baseline for random play.  This code should run in less than 30 seconds (ours runs in 2 seconds). 


In [ ]:
### This cell should produce from scratch a plot showing a learning curve for a single agent.


## Part 2A
 
Using the minimax policy you computed, answer the following question: The first player (Player 1) drops his/her first disk into column 2 (counting from the left). Consider the resulting state, shown in the following code cell. 

In [ ]:
env = connect.Connect(verbose=False)
env.reset(first_player='o')
env.act(action=1)
print(env.grid[::-1])

What is the Minimax Value of this state for Player 2? That is, assuming an optimal opponent, does the Minimax Policy expect to win the game (value = 1), lose the game (value = –1), or end the game in a draw (value = 0)? Please state your answer as a number.    

* The code cell below should compute this value and assign it to a variable called `state_value`.
* Count the number of branches of the game tree that were examined and assign this number to a variable called `num_branches`.

In [ ]:
### Write all your code for Part 2A in or above this cell.

# state_value = ...
# num_branches = ...


In [ ]:
# This is an autograded test cell. Do not delete or change, otherwise you will get 
# no marks for this part of the assignment. Please make sure that this cell has 
# access to the variables state_value and num_branches.

## Part 2B
Plot a learning curve similar to the one in Part 1, comparing your Q-learning algorithm, random play, and Minimax play. Assume as before that the opponent always plays first and uses a random policy.

In [ ]:
### Write all your code for Exercise 2 (B) in or above this cell.


Explain your findings in 3 or fewer sentences. Which policy is better? Why?

YOUR ANSWER HERE.



## Part 3

Using your algorithm, compute the value of the game for your player (recall: your player goes second against a random opponent). The code cell below should compute this value and assign it to a variable called `optimal_policy_value`.

In [ ]:
### Write all your code for Part 3 in or above this cell.

# optimal_policy_value = ...


In [ ]:
# This is an autograded test cell. Do not delete or change, otherwise you will get 
# no marks for this part of the assignment. Please make sure that this cell has 
# access to the variable optimal_policy_value.